In [3]:
import pandas as pd
from prophet import Prophet
import numpy as np

df = pd.read_csv('Sample_Superstore.csv', encoding='ISO-8859-1')
df['Order Date'] = pd.to_datetime(df['Order Date'])

forecast_days = 30
last_actual_date = df['Order Date'].max()

metrics = ['Sales', 'Profit', 'Quantity']
all_forecasts = []

for metric in metrics:
    daily = df.groupby('Order Date')[metric].sum().reset_index()
    daily.columns = ['ds', 'y']

    model = Prophet()
    model.fit(daily)

    future = model.make_future_dataframe(periods=forecast_days)
    forecast = model.predict(future)

    forecast_future = forecast[forecast['ds'] > last_actual_date]
    metric_df = forecast_future[['ds', 'yhat']].copy()
    metric_df['Metric'] = metric
    metric_df['yhat'] = metric_df['yhat'].round(2)

    all_forecasts.append(metric_df)

combined = pd.concat(all_forecasts, ignore_index=True)
forecast_data = combined.pivot(index='ds', columns='Metric', values='yhat').reset_index()
forecast_data.fillna(0, inplace=True)

forecast_formatted = pd.DataFrame({
    'Row ID': range(1, len(forecast_data)+1),
    'Order ID': ['FORECAST'] * len(forecast_data),
    'Order Date': forecast_data['ds'],
    'Ship Date': forecast_data['ds'] + pd.Timedelta(days=3),
    'Ship Mode': ['Forecast'] * len(forecast_data),
    'Customer ID': ['Forecast'] * len(forecast_data),
    'Customer Name': ['Forecast'] * len(forecast_data),
    'Segment': ['Forecast'] * len(forecast_data),
    'Country': ['United States'] * len(forecast_data),
    'City': ['Forecast'] * len(forecast_data),
    'State': ['Forecast'] * len(forecast_data),
    'Postal Code': [0] * len(forecast_data),
    'Region': ['Forecast'] * len(forecast_data),
    'Product ID': ['Forecast'] * len(forecast_data),
    'Category': ['Forecast'] * len(forecast_data),
    'Sub-Category': ['Forecast'] * len(forecast_data),
    'Product Name': ['Forecasted Data'] * len(forecast_data),
    'Sales': forecast_data.get('Sales', 0),
    'Quantity': forecast_data.get('Quantity', 0),
    'Discount': [0.0] * len(forecast_data),
    'Profit': forecast_data.get('Profit', 0)
})

historical_columns = forecast_formatted.columns
historical_formatted = df.copy()
historical_formatted = historical_formatted.reindex(columns=historical_columns, fill_value=0)

combined_data = pd.concat([historical_formatted, forecast_formatted], ignore_index=True)

combined_data.to_csv('superstore_historical_plus_forecast.csv', index=False)
print("Combined historical + forecast data saved to 'superstore_historical_plus_forecast.csv'")


16:31:06 - cmdstanpy - INFO - Chain [1] start processing
16:31:06 - cmdstanpy - INFO - Chain [1] done processing
16:31:06 - cmdstanpy - INFO - Chain [1] start processing
16:31:07 - cmdstanpy - INFO - Chain [1] done processing
16:31:07 - cmdstanpy - INFO - Chain [1] start processing
16:31:07 - cmdstanpy - INFO - Chain [1] done processing


Combined historical + forecast data saved to 'superstore_historical_plus_forecast.csv'
